# Reinforcement learning from (human) feedback

TRL - https://github.com/lvwerra/trl

In [1]:
 %pip install transformers trl wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 

In [2]:
%pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

In [4]:
from datasets import load_dataset

In [5]:
# тут задаются гиперпараметры
config = PPOConfig(
    model_name="ai-forever/rugpt3small_based_on_gpt2",
    learning_rate=1.41e-5,
    log_with=None,
    mini_batch_size=16
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

Для генерирования текстов тут используется toxic датасет https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments?select=labeled.csv
(с семинара)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/ВШЭ/Магистратура/NLP/labeled.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['comment', 'toxic'],
        num_rows: 14412
    })
})

In [29]:
dataset['train'][0]

{'comment': 'Верблюдов-то за что? Дебилы, бл...\n', 'toxic': 1.0}

In [30]:
def build_dataset(config,
                  dataset_name=dataset['train'],
                  input_min_text_length=2,
                  input_max_text_length=8):
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    ds = dataset
    ds = ds.rename_columns({"comment": "review"})

    # еще есть фильтрация по длине
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)
    ds = ds.filter(lambda x: len(x["review"]) < 2000, batched=False)

    # длина кусочка определяется случайно
    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [31]:
dataset = build_dataset(config)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
# active_model
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
# reference_model (обратите внимание что это одна и так же модель изначально)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [32]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'toxic', 'input_ids', 'query'],
        num_rows: 3454
    })
})

In [23]:
ppo_trainer = PPOTrainer(config=config, model=model, ref_model=ref_model, tokenizer=tokenizer,
                         dataset=dataset["train"], data_collator=collator)

In [24]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

Для оценки тональности используется пайплайн из huggingface и предобученная модель

In [33]:
sentiment_pipe = pipeline("sentiment-analysis", model="cointegrated/rubert-tiny-sentiment-balanced", device=device)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Для каждого текста она выдает скор негативности и положительности. Для обучения дальше будет использоваться скор положительности

In [35]:
text = "Это ужасный фильм!"
sentiment_pipe(text, **sent_kwargs)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'negative', 'score': 0.42893582582473755},
  {'label': 'neutral', 'score': -0.6796736121177673},
  {'label': 'positive', 'score': 0.11412131786346436}]]

In [36]:
text = "Отличный фильм!"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'negative', 'score': -3.383460760116577},
  {'label': 'neutral', 'score': -1.3034355640411377},
  {'label': 'positive', 'score': 4.374805450439453}]]

Еще одно место, где можно настраивать параметры. Эти параметры мы разбирали на семинаре по GPT они отвечают за генерацию.

In [37]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0,
              "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

Само обучение. Для небольших моделей оно даже не очень долгое

In [42]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader),
                         total=dataset["train"].num_rows//ppo_trainer.dataloader.batch_sampler.batch_size):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[2]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

  0%|          | 0/13 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_colwidth', 5000)

Можно посмотреть на результаты, сгенериров продолжения старой и новой модель

In [45]:
#### get a batch from the dataset
bs = 16
game_data = dict()
dataset['train'].set_format("pandas")
df_batch = dataset['train'][:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]




/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [46]:
pd.DataFrame.from_dict(game_data)

,query,response (before),response (after),rewards (before),rewards (after)
0,Коммунизм - это форма,"власти Советов, основывающаяся на","жизни, открытая для каждого) \n",3.021374,1.102060
1,Уважаемый,интеллектуал! Я с огромным удовольствием пере,господин программумер! 8. писатель,0.468644,4.128710
2,Ну да,", конечно, — ошарашенно прошептал Чарльз. —",. Мне даже обезьянка понравилась)\nВ тоже,-0.766515,0.071651
3,"Понимаете, тут","у меня случилась личная встреча. Я присев на стул, уставился на Гре",...пора брать и ухать...\n\n— — — — — —,1.332774,-1.172584
4,От неё,"мне сегодня нечего делать, — промолвила",только мне и жить хорошо). Это очень здорово,-0.767559,-0.728448
5,"Они не учли,",что девушки Патрик в военной форме не,что хорошо женщинам). - То есть,2.682849,0.620295
6,Блядь,кем мы были сегодня - сукиным сыном или бандитом?,"!"" \n Тепловатая и мягкая, как густой бук,",3.268764,-0.282972
7,Допустим гомикадзе действительно,что-то придумал? :)\n\n муниципального образования \n,хорошая). В общем велком всем:)\n\nЕсть,-0.088266,-0.982911
8,В заголовке какой-то,"личной беседы он упомянул, что не позволит мне с тобой",левый):-)Спасибо большое большое! Нравиться)))\n,0.312805,-0.524331
9,Могли абон,"ентские тарифы встать в один ряд с ценой пакетов, которую компания берет",енту сказать\nРастраталическля )\n\nУ меня лично,-0.080974,-1.006148


Вроде тексты положительнее, хотя "Уважаемый господин программумер!" довольно печально.